# Plot the Amtrak Thruway routes

Do these make sense?

Think about how to grab local buses that also travel the same origin / destination.

In [1]:
import geopandas as gpd
import intake
import pandas as pd

import _utils
from _utils import GCS_FILE_PATH, SELECTED_DATE
from shared_utils import geography_utils

catalog = intake.open_catalog("*.yml")

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
trip_gdf = gpd.read_parquet(
    f"{GCS_FILE_PATH}amtrak_thruway_trips_{SELECTED_DATE}.parquet")

gdf = _utils.aggregate_to_route(trip_gdf)

## Maps for Amtrak routes - see what OD pairs jump out

In [ ]:
def make_map(gdf: gpd.GeoDataFrame, route: str): 
    
    subset = gdf[gdf.route_id==route].reset_index(drop=True)
    
    print(f"Route: {route}    Route Type: {subset.route_type.iloc[0]}")

    display(subset[["origin_stop_name", "destination_stop_name"]].drop_duplicates())
    
    m = subset.explore("trip_id_nunique", categorical = False)
    display(m)
    
    display(subset)


In [ ]:
amtrak_routes = gdf[gdf.route_type=='3'].route_id.unique().tolist()

for r in amtrak_routes:
    make_map(gdf, r)

In [ ]:
amtrak_rail_routes = gdf[gdf.route_type=='2'].route_id.unique().tolist()

for r in amtrak_rail_routes:
    make_map(gdf, r)

## Typical Distance of Amtrak Thruway

There are some short distance ones. Where are these?

Note: These distances, because they're drawn by arraying points in a row, are "as the crow flies", and not actual driving distance. Distance on road network would be longer.

Very few < 10 miles, even < 20 miles. 

Can use higher distance cut-offs, then later go back and look for the trips that go from Emeryville - San Francisco.

In [3]:
gdf = gdf.assign(
    route_mi = ((gdf.geometry.to_crs(geography_utils.CA_StatePlane)
                 .length).divide(geography_utils.FEET_PER_MI)).round(2)
)

In [4]:
# What routes are the ones that have short distances?
# Most of them look long distance
for mile in [10, 20, 30]:
    subset = gdf[gdf.route_mi <= mile]
    
    print(f"Routes shorter than {mile} mi long")
    
    keep_cols = ["origin_stop_name", "destination_stop_name", "trip_id_nunique"]
    display(subset[keep_cols].drop_duplicates())

Routes shorter than 10 mi long


,origin_stop_name,destination_stop_name,trip_id_nunique
24,Stockton,Stockton,1
42,Emeryville,San Francisco,18
43,San Francisco,Emeryville,17


Routes shorter than 20 mi long


,origin_stop_name,destination_stop_name,trip_id_nunique
9,Roseville Amtrak,Sacramento,2
17,Davis,Sacramento,1
24,Stockton,Stockton,1
42,Emeryville,San Francisco,18
43,San Francisco,Emeryville,17


Routes shorter than 30 mi long


,origin_stop_name,destination_stop_name,trip_id_nunique
2,Carmel Amtrak Bus Stop,Salinas,2
8,Salinas,Carmel Amtrak Bus Stop,2
9,Roseville Amtrak,Sacramento,2
17,Davis,Sacramento,1
24,Stockton,Stockton,1
42,Emeryville,San Francisco,18
43,San Francisco,Emeryville,17
67,Martinez,Napa Amtrak Bus Stop,1
70,Napa Amtrak Bus Stop,Martinez,1


In [ ]:
# https://www.rome2rio.com/map/San-Jose/Stockton#r/Train/s/0

* MST runs longer routes
* Eastern Sierra Transit (Lancaster to Reno)
* Sonoma County Transit...or is it Golden Gate Transit 
* SolTrans

Also maybe grab a full week to get all the unique OD pairs

1. Follow Amtrak thruway (or same OD)
2. Don't follow Amtrak thruway but go longer distance
Longer route (> certain distance) get at Eastern Sierra
3. Long routes that travel on SHN on extended period of time